In [ ]:
# This is a script used to calculate the core genome Average Nucleotide Identity of different serotypes of E.coli
# Core genomes used in this analysis are Soft core genome defined by Roary
# Author: Huifeng Hu(hzauhhf@gmail.com)

# Please note: In this analysis, we want to analysis ANI of more than 6,000 genomes of E.coli. 
# And it is not possible to run Roary with over 6,000 genomes, so we run Roary with 1702 representative genomes and use Blast to find
# core genomes in other genomes

# THe analysis can be divied into following steps
# 1. Get sequence of core gene accoring Roary result(gene_presence_absence.csv)
# 2. Use CD-HIT to cluster each gene family and get representative gene sequences
# 3. Build a Blastn database with these representative sequences
# 4, Use orfs of 6,0000 genomes as query, search against the core gene database
# 5. Trim with Blast result and get the best hit result
# 6. FastANI was used to calculate pairs of core genomes(Please note parameter "--fragLen" must be set according your shortest core gene.)

In [ ]:
# 1. Get sequence of core gene accoring Roary result(gene_presence_absence.csv)
# We use gene id as key and get nucl sequence in gff files;
import os
gff_dir=r'path/to/gff_files'
id2fasta={}
for i in os.listdir(gff_dir):
    f=open(os.path.join(gff_dir,i),'r')
    data=f.read().split('##FASTA')
    f.close()
    dd={}
    for i in data[1].split('>')[1:]:
        dd[i.split('\n')[0]] = i.split('\n',1)[1].replace('\n','')
    for i in data[0].split('\n')[:-1]:
        if i[0]!= '#':
            id2fasta[i.split('\t')[-1].split(';')[0].split('=')[1]] = dd[i.split('\t')[0]][int(i.split('\t')[3])-1:int(i.split('\t')[4])]
import pandas as pd
f=pd.read_csv("gene_presence_absence.csv",low_memory=False)
ff=f[f['No. isolates']>1619] # Set according roary result
d={}
for index, row in ff.iterrows():
    d[list(row)[0]]=list(row)[14:]
out_put_path =r'path/to/your/core/gene/seq'
for i in d:
    of=open(os.path.join(out_put_path,i),'w')
    for j in d[i]:
        if type(j)==str:
            if '\t' in j:
                for k in j.split('\t'):
                    of.write('>'+k+'\n'+id2fasta[k]+'\n')
            else:
                of.write('>'+j+'\n'+id2fasta[j]+'\n')
    of.close()

In [ ]:
# 2. Use CD-HIT to cluster each gene family and get representative gene sequences
# 3. Build a Blastn database with these representative sequences
import os
for i in os.listdir(r'path/to/your/core/gene/seq'):
    if 'py' not in i:
        a='cd-hit -i '+i+' -o ../'+i+' -c 0.8 -d 50 -T 8 -M 0'
        os.system(a)
# 3. Build a Blastn database with these representative sequences        
os.system('cat *.representative sequences > core_genomes_DB.fasta')
# 4, Use orfs of 6,0000 genomes as query, search against the core gene database
os.system('makeblastdb -in core_genomes_DB.fasta -dbtype nucl')
os.system('blastn -query *.fna -db core_genomes_DB.fasta -evalue 1e-5 -out *.blast_result -outfmt 6 -num_threads 32')

In [ ]:
# 5. Trim with Blast result and get the best hit result
# We select best hit in Blast result and extract hit range of query sequences as core gene 
import os
def get_best_hit(input_file,output_file):
    f=open(input_file,'r')
    data=f.read().split('\n')
    f.close()
    besthit={}
    for i in data[:-1]:
        if i.split('\t')[0] not in besthit:
            besthit[i.split('\t')[0]] = i
        else:
            pass
    of=open(os.path.join(best_hit_path,output_file),'w')
    for i in besthit:
        of.write(besthit[i]+'\n')
    of.close()

blast_result_path=r'path/to/your/blast/result'
best_hit_path=r'path/to/your/best_hit'

for each_file in os.listdir(blast_result_path):
    get_best_hit(os.path.join(blast_result_path,each_file),each_file)

nucl_dir = r'path/to/your/nucl_file' 
core_genome_dir=r'path/to/your/core_genomes'

def get_core_genome(x):
    d={}
    f=open(os.path.join(nucl_dir,x+'.nucl'),'r')
    data=f.read().split('>',1)[1].split('\n>')
    f.close()
    for i in data:
        d[i.split('\n',1)[0].split(' ')[0]] = i.split('\n',1)[1].replace('\n','')
    f=open(os.path.join(best_hit_path,x+'_coregenome-hit.out'),'r')
    data=f.read().split('\n')[:-1]
    f.close()
    of=open(os.path.join(o_dir,x+'.fasta'),'w')
    for i in data:
        of.write('>'+i.split('\t')[0]+'\n'+d[i.split('\t')[0]]+'\n')
    of.close()
        
for i in os.listdir(hit_dir):
    get_core_genome(i.split('_core')[0])


In [ ]:
# 6. FastANI was used to calculate pairs of core genomes(Please note parameter "--fragLen" must be set according your shortest core gene.)
# fastANI were running as followings
fastANI -q query_geonomes.fasta -r reference_genomes.fasta -o query:refer -t 'n' --fragLen 100